In [1]:
import sys, os
import numpy as np
import adi
from pysmu import Session, Mode

In [2]:
sample_rate = int(1e6) # Hz
tone_freq = int(1e3) # Hz
center_freq = int(2.45e9) - tone_freq # Hz
n_samples = 2**12 # number of samples returned per call to rx()

sdr = adi.Pluto("ip:192.168.0.40")
sdr.sample_rate = sample_rate
sdr.tx_rf_bandwidth = sample_rate # filter cutoff, just set it to the same as sample rate
sdr.tx_lo = center_freq

sdr.gain_control_mode_chan0 = 'manual'
sdr.tx_hardwaregain_chan0 = -50 # Increase to increase tx power, valid range is -90 to 0 dB

sdr.tx_cyclic_buffer = True

In [3]:
session = Session()
 
if not session.devices:
    # no devices found
    print("No devices found!")
    sys.exit(1)

# single device operation
dev = session.devices[0]
# CH A voltage source
chan_a = dev.channels['A']
chan_a.mode = Mode.HI_Z
chan_b = dev.channels['B']
chan_b.mode = Mode.HI_Z

In [4]:
sdr.tx_hardwaregain_chan0 = -10 # valid range: -90 to 0 dB
t = np.linspace(0, 1, num=n_samples)
tx_samples = np.sin(2*np.pi*t*tone_freq)

sdr.tx_destroy_buffer()
sdr.tx(tx_samples)

samples = dev.get_samples(10)
samp_chan_a, samp_chan_b = zip(*samples)
v_chan_a, i_chan_a = zip(*samp_chan_a)
# v_chan_b, i_chan_b = zip(*samp_chan_b)
print("Voltage CH A: ", v_chan_a)
# print("Current CH A: ", i_chan_a)
# print("Voltage CH B: ", v_chan_b)
# print("Current CH B: ", i_chan_b)

Voltage CH A:  (2.490495204925537, 2.4905734062194824, 2.490729570388794, 2.4906513690948486, 2.490729570388794, 2.4905734062194824, 2.490495204925537, 2.4903390407562256, 2.4902608394622803, 2.490495204925537)
